In [ ]:
# Laura Chapman
# Script for developing statistical tools for Glue
# As of 6/12/18, correctly outputs mean, median, min, max as a printed table and a (badly formatted) popup window
# compute_statistic working after updating jupyter notebook

# Things to fix:
# Want statistical calculations to avoid accessing data arrays directly because this will be inefficient for large
# data sets- ideally with compute_statistic
# Make popup window well formatted with lined up columns and resizing with the width/height of the data

In [1]:
import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'

#start Glue
# app = GlueApplication(dc)

# imageviewer = app.new_data_viewer(ImageViewer)
# imageviewer.add_data(image)

# app.start()



In [2]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [15]:
# Prints the name of each data component and the mean, median, min, and max 
# Uses astropy.table to format the results
# Temporarily saves data array to memory, ideally would just be getting the values without this

mean_array = []
median_array = []
min_array = []
max_array = []
name_array = []
tables = []

for i in range (0, len(dc)):
    print(dc[i].label)
    print()
    for j in range (0, len(dc[i].components)):
        name = dc[i].components[j].label # Get the name of each component
        array = dc[i][name] # Access the data using the name
        name_array.append(name)
        mean_array.append(gua.nanmean(array))
        median_array.append(gua.nanmedian(array))
        min_array.append(gua.nanmin(array))
        max_array.append(gua.nanmax(array))
        headings = ('component','mean', 'median', 'min', 'max')
#         print(dc[i].components[j], gua.nanmean(array), gua.nanmedian(array), gua.nanmin(array), gua.nanmax(array))
        t = Table([name_array, mean_array, median_array, min_array, max_array], names = headings)
   
    # Save all the tables in an array for display in the popup
    tables.append(t)
    mean_array = [] # Clear the arrays
    median_array = []
    min_array = []
    max_array = []
    name_array = []
    print(t)
    print()
    
# print(tables)

w5_psc

   component          mean           median           min            max     
---------------- -------------- --------------- --------------- -------------
Pixel Axis 0 [x]         8885.0          8885.0             0.0       17770.0
         World 0         8885.0          8885.0             0.0       17770.0
              ID         8886.0          8886.0             1.0       17771.0
         RAJ2000     43.5318018       43.468829       41.081526     46.039649
         DEJ2000  60.3405947875       60.382988       59.246093     61.493241
            Jmag   13.563952446   13.7600002289   5.11999988556 17.4599990845
            Hmag  12.9239377975   13.1700000763    3.8900001049 16.4500007629
           Ksmag    12.66751194   12.9399995804   3.50999999046 16.0499992371
          __3.6_  12.5111446381   12.8000001907   4.86000013351 16.6200008392
          __4.5_  12.4496717453   12.7399997711   3.98000001907 16.1299991608
          __5.8_  12.3555679321   12.6499996185    2.140

In [18]:
# Modifying the above code to use compute_statistic to avoid accessing arrays of data directly

mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []
headings = ('component','mean', 'median', 'minimum', 'maximum', 'sum')

for i in range (0, len(dc)):
    print(dc[i].label)
    print()
    for j in range (0, len(dc[i].components)):
        name = dc[i].components[j].label # Get the name of each component
#         array = dc[i][name] # Access the data using the name
        name_array.append(name) # add to the name array to build the table
        mean_array.append(gua.compute_statistic('mean', dc[i].components[j], mask=None, axis=None, finite=True))
        median_array.append(gua.compute_statistic('median', dc[i].components[j], mask=None, axis=None, finite=True))        
        min_array.append(gua.compute_statistic('minimum', dc[i].components[j], mask=None, axis=None, finite=True))        
        max_array.append(gua.compute_statistic('maximum', dc[i].components[j], mask=None, axis=None, finite=True))        
        sum_array.append(gua.compute_statistic('sum', dc[i].components[j], mask=None, axis=None, finite=True))  
        
        
#         median_array.append(gua.nanmedian(array))
#         min_array.append(gua.nanmin(array))
#         max_array.append(gua.nanmax(array))

#         print(dc[i].components[j], gua.nanmean(array), gua.nanmedian(array), gua.nanmin(array), gua.nanmax(array))
        t = Table([name_array, mean_array, median_array, min_array, max_array, sum_array], names = headings)
   
    # Save all the tables in an array for display in the popup
    tables.append(t)
    mean_array = [] # Clear the arrays
    median_array = []
    min_array = []
    max_array = []
    name_array = []
    sum_array = []
    print(t)
    print()
    
# print(tables)

w5_psc



TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [7]:
import glue.utils.array as gua

In [20]:
# Test statement for compute_statistic

gua.compute_statistic('mean', np.asdc[0].components[0])

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [21]:
method_list = [func for func in dir(glue.utils.array) if callable(getattr(glue.utils.array, func))]

In [22]:
method_list

['_move_tuple_axes_first',
 'as_strided',
 'broadcast_to',
 'check_sorted',
 'coerce_numeric',
 'combine_slices',
 'compute_statistic',
 'find_chunk_shape',
 'format_minimal',
 'iterate_chunks',
 'nanmax',
 'nanmean',
 'nanmedian',
 'nanmin',
 'nansum',
 'pretty_number',
 'range',
 'shape_to_string',
 'stack_view',
 'unbroadcast',
 'unique',
 'view_shape']

In [23]:
import glue.core.data

In [24]:
method_list2 = [func for func in dir(glue.core.data) if callable(getattr(glue.core.data, func))]

In [25]:
method_list2

['Component',
 'ComponentID',
 'ComponentIDDict',
 'ComponentIDList',
 'ComponentLink',
 'ComponentReplacedMessage',
 'ComponentsChangedMessage',
 'CoordinateComponent',
 'CoordinateComponentLink',
 'Coordinates',
 'Data',
 'DataAddComponentMessage',
 'DataRemoveComponentMessage',
 'DataReorderComponentMessage',
 'DataUpdateMessage',
 'DerivedComponent',
 'ExternallyDerivableComponentsChangedMessage',
 'Hub',
 'IncompatibleAttribute',
 'NumericalDataChangedMessage',
 'OrderedDict',
 'PixelAlignedDataChangedMessage',
 'PixelComponentID',
 'SliceSubsetState',
 'Subset',
 'SubsetCreateMessage',
 'SubsetState',
 'VisualAttributes',
 'broadcast_to',
 'clear_cache',
 'compute_statistic',
 'contract',
 'datetime64_to_mpl',
 'iterate_chunks',
 'pixel_label',
 'settings',
 'split_component_view',
 'unbroadcast']

In [26]:
from glue.core.data import Data

In [27]:
# compute_statistic is working

gua.compute_statistic('mean', [1,2,3,5,66])

15.4

In [56]:
# Pop up window

LARGE_FONT= ("Verdana", 12)
NORM_FONT = ("Helvetica", 10)
SMALL_FONT = ("Helvetica", 8)

import tkinter as tk
from tkinter import ttk

In [71]:
# Creates a popup window with the statistical information
# Does not preserve the lined-up formatting of the astropy tables- to do

import tkinter as tk

num_cols = len(headings)
width = num_cols * 150

class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        self.hi_there = tk.Button(self)
        w = tk.Message(self, text=tables, width=width)
        w.pack()
#         self.hi_there["text"] = "Two houses both alike in dignity in fair verona where we lay our scene"
#         self.hi_there.pack(side="top")

        self.quit = tk.Button(self, text="Done", fg="red",
                              command=root.destroy)
        self.quit.pack(side="bottom")

root = tk.Tk()
root.geometry("400x400")
# root.geometry("500x100+300+300")
app = Application(master=root)
app.mainloop()